# Supporting Material for CARDIS 2020 Paper


This notebook is *NOT* designed to be full documentation on ChipJabber-BBI or similar tooling. This documentation is designed to show the work done in this paper, including all the fragile parts of it.

## Physical Setup

The majority of the work is described in the paper at XXXXXX. The following important notes are here:

* A DP832 is used to set the voltage for the BBI probe.
* A ChipWhisperer-Lite is used for triggering. A header is mounted on JP8, and the middle of the 3-pin header (pin 2) is used to trigger the BBI tool.
* Optionally, a PicoScope can be used to monitor the injected sigals (not required).

## Instrument Connection Scripts

The following contains various required scripts etc to connect to the instruments used. This will require you to install the following:

* chipwhisperer
* picoscope
* pyvisa


### ChipWhisperer Connection

In [ ]:
PLATFORM="CW308_STM32F4"
%run "../Helper_Scripts/Setup_Generic.ipynb"
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "enable_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.resetDcms()
scope.glitch.repeat = 5
scope.glitch.ext_offset = 260

In [ ]:
scope.glitch

In [ ]:
reset_target(scope)

In [ ]:
# Turn the glitch on/off by enabling glitch_lp - this assumes you mount a header on JP8,
# then use pin 2 of JP8 and route it to the BBI tool!
scope.io.glitch_lp = True

### DP832 - Power Supply

To control the ChipJabber-SimpleBBI probe, an external power supply is used. The authors had a DP832 on-hand and choose to use this, the following is required to control the power supply to adjust the voltage. You'll need to figure out the correct connect string for the USB part.

In [ ]:
import visa

class DP832(object):
    def __init__(self):
        pass

    def conn(self, constr='USB0::6833::3601::DP8C152800994::0::INSTR'):
        """Attempt to connect to instrument"""
        rm = visa.ResourceManager()
        self.inst = rm.open_resource(constr)

    def identify(self):
        """Return identify string which has serial number"""
        return self.inst.query("*IDN?")

    def readings(self, channel="CH1"):
        """Read voltage/current/power from CH1/CH2/CH3"""       
        resp = self.inst.query("MEAS:ALL? %s"%channel)
        resp = resp.split(',')
        dr = {"v":float(resp[0]), "i":float(resp[1]), "p":float(resp[2])}
        return dr
    
    def set_voltage(self, voltage, channel="CH1"):
        vstring = "%f"%voltage
        self.inst.write(":APPL " + channel + ", " + vstring)
        
    def set_output(self, enabled, channel="CH1"):
        if enabled:
            state = "ON"
        else:
            state = "OFF"
        self.inst.write(":OUTP " + channel + ", " + state)

    def dis(self):
        del self.inst

    def writing(self, command=""):
        self.inst.write(command)


In [ ]:
psu = DP832()
psu.conn()
psu.set_output(True, "CH1")
psu.set_voltage(3.0, "CH1")

## PicoScope PS6000 Connection

In [ ]:
#If you don't have a picoscope, run this cell and not the following ones
ps = None

In [ ]:
from picoscope import ps6000
try:
    ps.close()
except NameError:
    pass

ps = ps6000.PS6000()

In [ ]:
res = ps.setSamplingFrequency(1.25E9, 0.8E5)
sampleRate = res[0]
print("Sampling @ %f MHz, %d samples" % (res[0] / 1E6, res[1]))

ps.setChannel("A", "DC", 500E-3) #A has 10:1 probe on it, trigger, +/-5V range
ps.setChannel("B", "DC", 1) #B has 100:1 probe on it, voltage frmo injector, +/- 100V range
ps.setChannel("C", "DC50", 1) #C has current probe on it, /5 for A reading
ps.setChannel("D", "DC", 500E-3) #D has 10:1 probe on it, voltage of target on it

ps.setSimpleTrigger("A", 0.1, 'Rising', timeout_ms=500, enabled=True)

In [ ]:
def set_voltage(volt):
    
    #If picoscope variable exists, change settings, otherwise we ignore it
    if ps:    
         #B has 10:1 probe on it, voltage frmo injector
        if volt > 1.6:
            ps.setChannel("B", "DC", 5)
        elif volt > 0.6:    
            ps.setChannel("B", "DC", 2)
        else:
            ps.setChannel("B", "DC", 1)


        if volt > 3:
            ps.setChannel("C", "DC50", 5)
        elif volt > 2:
            ps.setChannel("C", "DC50", 2) #C has current probe on it, /5 for A reading
        elif volt > 1:
            ps.setChannel("C", "DC50", 1)
        else:
            ps.setChannel("C", "DC50", 500E-3)
        
        
    psu.set_voltage(volt, "CH1")

## Building HW-AES Attack Demo Firmware

ChipWhisperer 'target device' firmware is modified for the WLCSP device. The attached requires you to copy the included new HAL to the correct location (forked CW firmware used for now, if a PR is submitted could be merged in presumably)

In [ ]:
PLATFORM="CW308_STM32F4"

In [ ]:
%%bash
cd ../../hardware/victims/simpleserial-aes
make PLATFORM="CW308_STM32F4" CRYPTO_TARGET="HWAES" STM32F4_WLCSP="1"

### CPA Attack for Reference

In [ ]:
fw_path = "simpleserial-aes-CW308_STM32F4.hex"
cw.program_target(scope, prog, fw_path)

In [ ]:
project = cw.create_project("STM32F415OG_AES.cwp", overwrite=True)

#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
N = 15000  # Number of traces
scope.adc.samples=2000

scope.gain.db = 38


for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

print(scope.adc.trig_count)
project.close()

In [ ]:
project = cw.open_project("STM32F415OG_AES")

In [ ]:
import chipwhisperer.common.api.lascar as cw_lascar
from lascar import *
cw_container = cw_lascar.CWContainer(project, project.textouts, start=0, end=None) #optional start and end args set start and end points for analysis
guess_range = range(256)

In [ ]:
leakage = cw_lascar.lastround_HD_gen

In [ ]:
cpa_engines = [CpaEngine("cpa_%02d" % i, leakage(i), guess_range) for i in range(16)]
session = Session(cw_container, engines=cpa_engines).run(batch_size=50)

In [ ]:
import chipwhisperer.analyzer as cwa
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []

last_round_key = cwa.aes_funcs.key_schedule_rounds(list(project.keys[0]),0,10)

for i in range(16):
    results = cpa_engines[i].finalize()
    #results = results[:,1110:1140]
    xrange = range(len(results[0xD0]))
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    p.line(xrange, results[guess], color="black")
    p.line(xrange, results[last_round_key[i]], color="red")
    key_guess.append(guess)
    
show(p)

In [ ]:
last_round_key = cwa.aes_funcs.key_schedule_rounds(list(project.keys[0]),0,10)
disp = cw_lascar.LascarDisplay(cpa_engines, last_round_key)
disp.show_pge()

## Testing FLASH Memory Read/Write

We can use the serial bootloader to dump flash memory. This is much slower than JTAG, but doesn't require other hardware. This was partially used in exploring the flash memory corruption issue (however a faster JTAG connection was also used later).

The following section of code isn't required, but is included for completeness of the work.

In [ ]:
import time
scope.io.target_pwr = False
time.sleep(1)
scope.io.target_pwr = True

stm = prog()
stm.scope = scope
stm.open()
stm.find()

In [ ]:
#Erase memory only
stm32f.cmdEraseMemory()


#Erase & Program memory (disconnects after as well) - uncomment this
#fw_path = "simpleserial-aes-CW308_STM32F4.hex"
#cw.program_target(scope, prog, fw_path)

In [ ]:
stm32f = stm.stm32prog()

bits_prog = 0
bytes_off = 0

for i in range(0, 0xFFFFF, 0x1000):
    data = stm32f.cmdReadMemory(0x8000000 + i, 256) #Read an example block

    chunk_err = False
    
    for d in data:
        diff = d ^ 0xff
        if diff:
            chunk_err = True
            bytes_off += 1
            bits_prog += bin(diff).count('1')
            
    if chunk_err:
        print("%x --> Incorrect bits"%(i))
            
        
print(bits_prog)
print(bytes_off)

In [ ]:
for i in range(0, 50000, 256):
    print(data)
    data = stm32f.cmdReadMemory(0x8000000+i, 256)
    if 0 not in data:
        break
            
print(i)

In [ ]:
stm.close()

In [ ]:
scope.io.pdic = None

In [ ]:
import time
scope.io.target_pwr = True
scope.glitch.output = "enable_only"
#psu.set_voltage(28)

scope.glitch.repeat = 150
scope.glitch.trigger_src = "manual"

for i in range(0, 5000):
    scope.io.glitch_hp = True
    scope.glitch.manual_trigger()
    scope.io.glitch_hp = False
    
    #time.sleep(0.05)

scope.io.target_pwr = False

In [ ]:
data = stm32f.cmdReadMemory(0x8003000, 0x100)
print(data)

In [ ]:
data4 = stm32f.cmdReadMemory(0x8000000, 0x100)
print(data4)

In [ ]:
#Test board #1

# Erase - then re-run super-loop
#data3 = stm32f.cmdReadMemory(0x8000000, 0x100)
print(data3)

In [ ]:
#Test board #1

#Testing after a serious FI campaign...
#data1 = stm32f.cmdReadMemory(0x8000000, 0x100)
#data2 = stm32f.cmdReadMemory(0x8000100, 0x100)
print(data1)
print(data2)

### Capturing super-intense pulse waveform

The following will generate a crazy pulse waveform by hammering the glitch output.

In [ ]:
scope.io.target_pwr = True


ps.setSimpleTrigger("B", 0.5, 'Rising', timeout_ms=500, enabled=True)

set_voltage(25)
scope.glitch.repeat = 50
scope.glitch.trigger_src = "manual"

ps.runBlock(pretrig=0.1)

scope.glitch.manual_trigger()

scope.io.target_pwr = False

ps.waitReady()
injector_data, current_data = get_psdata(ps)


#Set back to default
ps.setSimpleTrigger("A", 0.1, 'Rising', timeout_ms=500, enabled=True)

In [ ]:
ps.close()

## Basic Glitch Loop

The following is used when testing the "double-loop" glitch work. This is used for the majority of analysis effort in this paper.

### Building Glitch Firmware

We use the ChipWhisperer 'glitch' firmware which implements the double-loop for us. We just need to specify we're using the WLCSP option of the STM32F4 target for the firmware to work (hopefully). Note the STM32F4_WLCSP option was added later, be sure you see `-DSTM32F4_WLCSP` based to the compiler.

If you don't specify WLCSP, the software builds & runs. However the trigger pins are different, so you'll get a bunch of 'trigger timeout' messages.

In [ ]:
%%bash
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM="CW308_STM32F4" CRYPTO_TARGET="NONE" STM32F4_WLCSP="1"

In [ ]:
fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
print(fw_path)
cw.program_target(scope, prog, fw_path)

In [ ]:
# test_mode 1 or test_mode 2 specifies the clock frequency of the target.
# To go above 40 Mhz you need to change flash timing settings in the target, which we didn't do
# to keep the same binary between tests.

test_mode = 1

#Test 1 - 7.37 MHz device clock
if test_mode == 1:
    scope.clock.adc_src = "clkgen_x4"
    scope.clock.clkgen_freq = 7372800
    target.baud = 38400

elif test_mode == 2:
    scope.clock.adc_src = "clkgen_x1"
    scope.clock.clkgen_freq = 40E6
    target.baud = (scope.clock.clkgen_freq / 7372800) * 38400
    
scope.glitch.resetDcms()

Run the following cell - you should see an output like this:

    {'valid': False, 'payload': None, 'full_response': 'ÿ\x92ÿrRESET   \nrC4090000\nz00\n', 'rv': None}
    
There may be other garbage first, but if the cell is working you'll see some sort of valid response.

In [ ]:
reset_target(scope)
set_voltage(0)
scope.io.glitch_lp = False
scope.arm()
target.write("g\n")
ret = scope.capture()
val = target.simpleserial_read_witherrors('r', 4) #For loop check
print(val)
scope.io.glitch_lp = True

### Running Glitch Experiments

Re-run the following cell when you want to clear the statistics.

In [ ]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["voltage", "width"])
gc.display_stats()

In [ ]:
from tqdm import tqdm_notebook
import struct

# Init target - power cycle, do reset, flush all old garbage from serial buffers
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True
time.sleep(0.2)
reset_target(scope)
target.flush()

# Use a fixed offset from the trigger - we are not experimenting with time location here.
scope.glitch.ext_offset = 200

#Use 100mV for sciencing
mv_step_size = 100

#Use higher value when dialing in stuff to run fast
#mv_step_size = 500

repeat_step_size = 1
repeat_max = 100

for mv_setting in tqdm_notebook(range(100, 20000, mv_step_size)):
    v_setting = mv_setting / 1000.0
    set_voltage(v_setting)
    for trial in range(0, 10):
        resets_in_a_row = 0
        for repeat in range(1, repeat_max, repeat_step_size):
            
            scope.glitch.repeat = repeat
            
            #As we sweep repeats upwards, don't test as far in the future
            if resets_in_a_row > 5:
                repeat_max = repeat
                break
            
            if scope.adc.state:
                # can detect crash here (fast) before timing out (slow)
                print("Trigger still high!")            
            
                reset_target(scope)
                target.flush()
                
                resets_in_a_row += 1
                
            scope.arm()

            #Do glitch loop
            target.write("g\n")
            ret = scope.capture()
            val = target.simpleserial_read_witherrors('r', 4)#For loop check

            if ret:
                print('Timeout - no trigger')
                gc.add("reset", (v_setting, scope.glitch.repeat))
                
                resets_in_a_row += 1
                
                #Device is slow to boot?
                reset_target(scope)
                target.flush()

            else:
                if val['valid'] is False:
                    gc.add("reset", (v_setting, scope.glitch.repeat))
                    resets_in_a_row += 1
                else:
                    
                    resets_in_a_row = 0

                    #print(val['payload'])

                    #gcnt = struct.unpack("<b", val['payload'])[0] #for code-flow check
                    gcnt = struct.unpack("<I", val['payload'])[0]

                    #print(gcnt)                
                    # for table display purposes
                    #if gnt != 0: #for code-flow check
                    if gcnt != 2500: #for loop check
                        gc.add("success", (v_setting, scope.glitch.repeat), metadata=val['payload'])
                        print(val['payload'])
                        print("🐙", end="")
                    else:
                        gc.add("normal", (v_setting, scope.glitch.repeat))           

In [ ]:
import numpy as np
# Saved as "location 1" on chip - these locations are arbitrary
if test_mode == 1:    
    #np.save("glitchloop_7_37mhz_location1_2_20V.npy", gc.results)
    #gr_737mhz = copy.deepcopy(gc.results)
    pass

if test_mode == 2:
    #np.save("glitchloop_40mhz_location1_1_20V_dynamic.npy", gc.results)
    #gr_40mhz = copy.deepcopy(gc.results)
    pass

In [ ]:
%matplotlib notebook
gc.results.plot_2d(plotdots={"success":".g", "reset":"xr", "normal":""})

In [ ]:
%matplotlib notebook
gr_737mhz.plot_2d(plotdots={"success":".g", "reset":"xr", "normal":""})

## MBED TLS Example

The following performs a fault on a MBED-TLS implementation.

This follows the ChipWhisperer RSA DFA tutorial - see that for more details of everything done here.

Currently the additional verification is skipped.

In [ ]:
scope.clock.adc_src = "clkgen_x4"
scope.clock.clkgen_freq = 7372800
target.baud = 38400

In [ ]:
%%bash
cd ../../hardware/victims/firmware/simpleserial-rsa
make PLATFORM="CW308_STM32F4" CRYPTO_TARGET="MBEDTLS" CRYPTO_OPTIONS="RSA" OPT="2" STM32F4_WLCSP="1"

In [ ]:
scope.clock.adc_src = "clkgen_x1"

In [ ]:
import time
fw_path = "../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
#reset_target(scope)
target.baud = 38400
scope.io.glitch_hp = False
scope.io.nrst = None
set_voltage(0)
time.sleep(1)
target.flush()
scope.arm()
target.write("t\n")

ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')
    
time.sleep(4)
output = target.read(timeout=10)
print(output)

In [ ]:
print("Total RSA Signature time was %d cycles"%scope.adc.trig_count)

In [ ]:
from Crypto.Hash import SHA256
from binascii import unhexlify, hexlify

def build_message(m, N):
    sha_id = "3031300d060960864801650304020105000420"
    N_len = (len(bin(N)) - 2 + 7) // 8
    pad_len = (len(hex(N)) - 2) // 2 - 3 - len(m)//2 - len(sha_id)//2
    padded_m = "0001" + "ff" * pad_len + "00" + sha_id + m
    return padded_m

hash_object = SHA256.new(data=b"Hello World!")
hashed_m = hexlify(hash_object.digest()).decode()
padded_m = build_message(hashed_m, N)
print(hashed_m)
print(padded_m)

In [ ]:
import gmpy2
from gmpy2 import mpz
from tqdm import tnrange
import time

N = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
e = 0x10001

#Settings frmo earlier
set_voltage(5.0)
scope.glitch.repeat = 10
scope.io.glitch_hp = True

glitch_list = []

#Scanning not really needed, doesn't hurt though
for i in tnrange(7000000, 7000100):
    scope.glitch.ext_offset = i
    target.flush()
    scope.arm()
    target.write("t\n")
    
    status = {'status':'', 'output':''}
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    time.sleep(3)
    
    # Read back signature
    output = target.read(timeout=5)
    print(output)
    if "4F09799" not in output:
        # Something abnormal happened
        if len(output) > 0:
            # Possible glitch!
            print("Possible glitch at offset {}\nOutput: {}".format(scope.glitch.ext_offset, output))
            status['status'] = 'invalid_notexploitable'
            
            # Get rest of signature back
            target.write("1\n")
            time.sleep(0.2)
            output += target.read(timeout=10)
            
            target.write("2\n")
            time.sleep(0.2)
            output += target.read(timeout=10)
            
            # Strip out extra simpleserial stuff
            newout = output.replace("r", "").replace("\nz00","").replace("\n","")
            print("Full output: {}".format(newout))
            if (len(newout) == 256) and "r0001F" not in output:
                print("Possible glitch!")
                
                sig = unhexlify(newout)

                sig_int = mpz(int.from_bytes(sig, "big"))
                m_int = mpz(int.from_bytes(unhexlify(padded_m), "big"))
                p_test = gmpy2.gcd(sig_int**e - m_int, N)

                real_p = "0xC36D0EB7FCD285223CFB5AABA5BDA3D82C01CAD19EA484A87EA4377637E75500FCB2005C5C7DD6EC4AC023CDA285D796C3D9E75E1EFC42488BB4F1D13AC30A57".lower()
                real_q = "0xC000DF51A7C77AE8D7C7370C1FF55B69E211C2B9E5DB1ED0BF61D0D9899620F4910E4168387E3C30AA1E00C339A795088452DD96A9A5EA5D9DCA68DA636032AF".lower()
                if(hex(p_test) == real_p) or (hex(p_test) == real_q):
                    print("Success - recovered actual key! Counting one for the good guys.")
                    status['status'] = 'invalid_exploitable'
        else:
            # Crash, reset and try again
            print("Probably crashed at {}".format(scope.glitch.ext_offset))
            status['status'] = 'reset'
            reset_target(scope)
            time.sleep(0.5)
    else:
        status['status'] = 'normal'
        
    glitch_list.append(status)

In [ ]:
reset = 0
normal = 0
exploit = 0
other = 0
total = len(glitch_list)
for i in glitch_list:
    if i['status'] == 'reset':
        reset += 1
    if i['status'] == 'normal':
        normal += 1
    if i['status'] == 'invalid_exploitable':
        exploit += 1
    if i['status'] == 'invalid_notexploitable':
        other += 1
        
print("Reset: %.2f %%"%(reset*100 / total))
print("Normal: %.2f %%"%(normal*100 / total))
print("Invalid: %.2f %%"%(other*100 / total))
print("Exploit: %.2f %%"%(exploit*100 / total))
        
#np.save("rsa_results.npy", glitch_list)

## DFA on AES

In [ ]:
#fw_path = "../../hardware/victims/firmware/simpleserial-aes/simeplserial-aes-{}.hex".format(PLATFORM)
fw_path = "simpleserial-aes-CW308_STM32F4.hex"
cw.program_target(scope, prog, fw_path)

In [ ]:
set_voltage(0.1)

In [ ]:
reset_target(scope)

In [ ]:
from Crypto.Cipher import AES

def run_encryption(text, key, ps=None):
    
    scope.io.glitch_hp = False    
    
    target.set_key(key=bytes(key), ack=True)
    
    scope.arm()
    target.simpleserial_write('p', bytes(text))
    res = target.simpleserial_read_witherrors('r', 16)
    scope.capture()
    golden = res['payload']
    
    cipher = AES.new(bytes(key), AES.MODE_ECB)
    expected = list(cipher.encrypt(bytes(text)))
    
    if list(golden) != list(expected):
        print(list(golden))
        print(list(expected))
        reset_target(scope)
        run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])
        run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])
        run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])
        raise IOError("Expected & Golden mis-match.")
        
    
    if ps:
        ps.runBlock()
    
    scope.io.glitch_hp = True
    
    scope.arm()
    target.simpleserial_write('p', bytes(text))
    res = target.simpleserial_read_witherrors('r', 16)
    scope.capture()
    
    # Reset
    if res['valid'] == False:       
        reset_widget.value += 1
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = True
        time.sleep(0.05)
        target.write("xxxxx\n")
    
    else:
        if res['payload'] != golden:
            #glitch?
            print(res['payload'])
            
    scope.io.glitch_hp = False
    
    return res
  

run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])

In [ ]:
import ipywidgets as widgets
normal_widget = widgets.IntText (value=0, description="Normal", disabled=True)
reset_widget = widgets.IntText (value=0, description="Reset", disabled=True)
glitch_widget = widgets.IntText (value=0, description="Glitch", disabled=True)

display(normal_widget)
display(reset_widget)
display(glitch_widget)

In [ ]:
from tqdm import tqdm_notebook
text = [0]*16
key = list(range(0, 16))

normal_widget.value = 0
reset_widget.value = 0
glitch_widget.value = 0

reset_target(scope)
run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])
run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])
run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])

trials = []

#Initial test done over 2.0 - 10.0V range, 0.1V step
#Next test done over 10.0 - 19.0V range, 0.5V step
for mv_setting in tqdm_notebook(range(2000, 10000, 100)):
    set_voltage(mv_setting/1000.0)
    for trial in range(0, 10):
        for offset in range(250, 280):
            scope.glitch.ext_offset = offset
            trial = {'psu':mv_setting/1000.0, 'offset':offset, 'result':None}
            res = 'FAIL'
            try:                
                res = run_encryption(text, key)
            except OSError as e:
                print(e)
                
            trial['result'] = res
            
            trials.append(trial)
                

In [ ]:
import numpy as np
#trials_partial_result = trials[:]
#np.save("aes_dfa_trials_july12_2020_19V.npy", trials)

In [ ]:
from picoscope import ps6000
from tqdm import tqdm_notebook

text = [0]*16
#key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]
key = [0]*16

normal_widget.value = 0
reset_widget.value = 0
glitch_widget.value = 0

reset_target(scope)
run_encryption([0]*16, [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6,	0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c])

for i in tqdm_notebook(range(0, 1)):
    run_encryption(text, key, ps)
    ps.waitReady()
    injector_data, current_data = get_psdata(ps)

In [ ]:
print(ps.getTriggerTimeOffset())

## Volt/Current Measurement Functions

In [ ]:
import numpy as np

def get_psdata(ps):
    trigger_data = ps.getDataV("A") * 10.0
    injector_data, injover = ps.getDataV("B", returnOverflow=True)
    current_data, currover = ps.getDataV("C", returnOverflow=True)
    vcc_data = ps.getDataV("D")
                                          
    injector_data *= 100.0 #1:100 probe
    current_data /= -5.0 #CT-6 probe uses 5mV = 1mA conversion, probe is installed "backwards" so we give negative sign
    
    if injover:
        print("WARNING: Injector (chb) overflow - need to adjust hardware range")
 
    if currover:
        print("WARNING: Current (chc) overflow - need to adjust hardware range")

    max_v_loc = np.argmax(abs(injector_data))
    max_i_loc = np.argmax(abs(current_data))
    
    max_v = injector_data[max_v_loc]
    max_i = current_data[max_i_loc]
    
    print("Pulse: %f V, %f mA [DBG: %d %d]"%(max_v, max_i*1000, max_v_loc, max_i_loc))
    
    return injector_data, current_data

In [ ]:
import matplotlib.pylab as plt
fig, ax1 = plt.subplots()

def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    adjust_yaxis(ax2,(y1-y2)/2,v2)
    adjust_yaxis(ax1,(y2-y1)/2,v1)

def adjust_yaxis(ax,ydif,v):
    """shift axis ax by ydiff, maintaining point v at the same location"""
    inv = ax.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, ydif))
    miny, maxy = ax.get_ylim()
    miny, maxy = miny - v, maxy - v
    if -miny>maxy or (-miny==maxy and dy > 0):
        nminy = miny
        nmaxy = miny*(maxy+dy)/(miny+dy)
    else:
        nmaxy = maxy
        nminy = maxy*(miny+dy)/(maxy+dy)
    ax.set_ylim(nminy+v, nmaxy+v)

color = 'tab:blue'
ax1.set_xlabel('Sample')
ax1.set_ylabel('Voltage (V)', color=color)
ax1.plot(injector_data[43000:48000], color=color)
#ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Current (mA)', color=color)  # we already handled the x-label with ax1
ax2.plot(current_data[43000:48000]*1000, color=color)
#ax2.tick_params(axis='y', labelcolor=color)

align_yaxis(ax1, 0, ax2, 0)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


In [ ]:
ps.close()

In [ ]:
scope

## Characterizing Drive Voltage vs. Set Current

In [ ]:
import time

scope.io.nrst = False

drive_results = []

for mv_setting in tqdm_notebook(range(100, 15000, 100)):   
    set_voltage(mv_setting / 1000.0)
    #Let power stabilize
    time.sleep(0.25)
    
    scope.io.target_pwr = True
    time.sleep(0.5)
    
    ps.runBlock()
    scope.io.glitch_hp = True    
    scope.arm()
    
    scope.io.tio4 = True
    scope.io.tio4 = False
    
    scope.capture()
    
    ps.waitReady()
    scope.io.target_pwr = False
    print(mv_setting / 1000.0)
    injector_data, current_data = get_psdata(ps)
    
    res = {'psu':mv_setting / 1000.0, 'injector_voltage':injector_data[:], 'injector_current':current_data[:]}
    
    drive_results.append(res)

In [ ]:
import numpy as np

#Commented out to avoid messing up stuff by accident!
#np.save("drive_results.npy", drive_results)

In [ ]:
#Release target afterwards

scope.io.tio4 = None
scope.io.target_pwr = False
time.sleep(0.5)
scope.io.target_pwr = True
scope.io.nrst = None

In [ ]:
#Read data back to check
drive_results_read = np.load("drive_results.npy", allow_pickle=True)

psu_setting = []
max_vol = []
max_cur = []

#Build array to plot
for i in range(0, len(drive_results_read)):
    result = drive_results_read[i]
    psu_setting.append(result['psu'])
    max_vol.append(max(abs(result['injector_voltage'])))
    max_cur.append(max(abs(result['injector_current']))) 
    
    print("Index %d = %f"%(i, result['psu']))

In [ ]:
import matplotlib.pylab as plt
plt.figure()
plt.plot(psu_setting, max_vol)
plt.title('Injection Voltage')

plt.figure()
plt.plot(psu_setting, max_cur)
plt.title('Injection Current')

plt.figure()
plt.plot(drive_results_read[29]['injector_voltage'][44000:48000])

plt.figure()
plt.plot(drive_results_read[29]['injector_current'][44000:48000])

plt.figure()
plt.plot(drive_results_read[100]['injector_voltage'][44000:48000])

plt.figure()
plt.plot(drive_results_read[100]['injector_current'][44000:48000])